In [0]:
#yanking this primo code from Shandy.
import csv
from os import listdir
from random import randint
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import time

POP_SONGS_PATH_TOP = '/content/drive/My Drive/audio/NonProg/TopOfThePops/Top Of The Pops/'
POP_SONGS_PATH_OTHER = '/content/drive/My Drive/audio/NonProg/Other Songs/'
POP_SONGS_PATH_ADDITIONAL_POP_SONGS = '/content/drive/My Drive/audio/NonProg/Additional Pop Songs/'
PROG_SONGS_PATH = '/content/drive/My Drive/audio/Prog/Progressive Rock Songs/'
SONG_SEGMENT_SIZE = 240 # in seconds
POP_CSV_FILE_NAME = '/content/drive/My Drive/audio/popMissingSongsFormatted.csv'
PROG_CSV_FILE_NAME = '/content/drive/My Drive/audio/progTempoAVGandVAR.csv'
FULL_POP_CSV_FILE_NAME = '/content/drive/My Drive/audio/fullPop.csv'

VALIDATION_SET_PROG = '/content/drive/My Drive/audio/Validation Set/Prog/'
VALIDATION_SET_POP = '/content/drive/My Drive/audio/Validation Set/Non-Prog/'
VALIDATION_CSV_FILENAME = '/content/drive/My Drive/audio/validation_set.csv'

NUM_SONGS = 2
SEGMENT_LENGTH_SEC = 20

from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive/')

# Get list of pop songs
pop1 = listdir(POP_SONGS_PATH_TOP)
pop2 = listdir(POP_SONGS_PATH_OTHER)
pop3 = listdir(POP_SONGS_PATH_ADDITIONAL_POP_SONGS)

vprog = listdir(VALIDATION_SET_PROG)
vpop  = listdir(VALIDATION_SET_POP)

#popSongs = listdir(POP_SONGS_PATH)
progSongs = listdir(PROG_SONGS_PATH)

#print(popSongs)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
#super cute logging function
def log(genre, songs, avg, var):
  if genre == "POP":
    #with open(POP_CSV_FILE_NAME, "a", newline='') as csvfile:
    with open(VALIDATION_CSV_FILENAME, "a", newline='') as csvfile:
        writer = csv.writer(csvfile)
        for i in range(len(songs)):
            writer.writerow([genre, songs[i], avg[i], var[i]])
    print(genre + " songs logged in csv file...")
  else:
    if genre == "PROG":
      with open(PROG_CSV_FILE_NAME, "a", newline='') as csvfile:
          writer = csv.writer(csvfile)
          for i in range(len(songs)):
              writer.writerow([genre, songs[i], avg[i], var[i]])
      print(genre + " songs logged in csv file...")
  return

#extract the tempo and var for a particular song
def get_avg_and_var_tempo(song_data, sample_rate):
  tempo = []
  beat_frames = []
  num_slice_samples = SEGMENT_LENGTH_SEC*sample_rate
  num_song_segments = int(np.floor(song_data.shape[0]/(num_slice_samples)))
  print("number of song segments is  = ", num_song_segments)
  for i in range(0,num_song_segments):
    tempo_slice, beat_frames_slice = librosa.beat.beat_track(y=song_data[i*num_slice_samples: i*num_slice_samples+num_slice_samples], sr=sample_rate)
    tempo.append(tempo_slice)
    beat_frames.append(beat_frames_slice)
  tempo = np.array(tempo)
  librosa.beat.beat_track(y=song_data, sr=sample_rate)
  return np.mean(tempo), np.var(tempo)

In [0]:
#code to actually run the algorithm to extract the feature

pop1_songs = [POP_SONGS_PATH_TOP + song for song in pop1]
pop2_songs = [POP_SONGS_PATH_OTHER + song for song in pop2]
pop3_songs = [POP_SONGS_PATH_ADDITIONAL_POP_SONGS + song for song in pop3]
vprog_songs= [VALIDATION_SET_PROG + song for song in vprog]
vpop_songs = [VALIDATION_SET_POP + song for song in vpop]
popSongs = []

popSongs = pop1_songs + pop2_songs + pop3_songs
#print(pop3_songs)
#for i in range(15,0,-1):
#  popSongs.append(pop3_songs[-i])
#popSongs.append(POP_SONGS_PATH_OTHER + "rasme-asheghi.mp3")
#print(popSongs)
#print(len(popSongs))
fullPopOut = []
songNames = []
#print(progSongs)

# Select a sample of 10 pop songs
#pop_samples = []
#prog_samples = []
#for _ in range(NUM_SONGS):
#    pop_samples.append(popSongs[randint(0, len(popSongs) - 1)])
#    prog_samples.append(progSongs[randint(0, len(progSongs) - 1)])

#build validation set features
#popSongs = vprog_songs + vpop_songs

avgList = []
varList = []

#for song in pop_samples:
for i, song in enumerate(popSongs):
  print(i, song)
  song_data , sample_rate = librosa.load(song)
  
  #seperate harmonic from percusive
  harmonic, percussive = librosa.effects.hpss(song_data)
  
  #get tempo avg and variance containers
  avg, var = get_avg_and_var_tempo(percussive, sample_rate)
  avgList.append(avg)
  varList.append(var)
  fullPopOut.append([song.split('/')[-1], avg, var])
  songNames.append(song.split('/')[-1])
  print(fullPopOut[-1])

np.save("/content/drive/My Drive/audio/fullpopAvgListOut2.npy",avgList)
np.save("/content/drive/My Drive/audio/fullpopVarListOut2.npy",varList)
log("POP", songNames, avgList, varList)
np.save("/content/drive/My Drive/audio/validationBackup.npy", np.array(fullPopOut))
log("POP", pop_samples, avgList, varList)

avgList = []
varList = []
#for song in prog_samples:
for song in progSongs:
  if song[-3:] != "mp3":
    continue
  print(PROG_SONGS_PATH + song)
  song_data , sample_rate = librosa.load(PROG_SONGS_PATH + song)
  
  #seperate harmonic from percusive
  harmonic, percussive = librosa.effects.hpss(song_data)
  
  #get tempo avg and variance containers
  avg, var = get_avg_and_var_tempo(percussive, sample_rate)
  avgList.append(avg)
  varList.append(var)
np.save("/content/drive/My Drive/audio/progAvgListOut2.npy",avgList)
np.save("/content/drive/My Drive/audio/progVarListOut2.npy",varList)
log("PROG", progSongs, avgList, varList)
#log("PROG", prog_samples, avgList, varList)



'avgList = []\nvarList = []\n#for song in prog_samples:\nfor song in progSongs:\n  if song[-3:] != "mp3":\n    continue\n  print(PROG_SONGS_PATH + song)\n  song_data , sample_rate = librosa.load(PROG_SONGS_PATH + song)\n  \n  #seperate harmonic from percusive\n  harmonic, percussive = librosa.effects.hpss(song_data)\n  \n  #get tempo avg and variance containers\n  avg, var = get_avg_and_var_tempo(percussive, sample_rate)\n  avgList.append(avg)\n  varList.append(var)\nnp.save("/content/drive/My Drive/audio/progAvgListOut2.npy",avgList)\nnp.save("/content/drive/My Drive/audio/progVarListOut2.npy",varList)\nlog("PROG", progSongs, avgList, varList)'

In [0]:
#generate plot data

'''progVar = np.load("/content/drive/My Drive/audio/varListOut.npy")
progAvg = np.load("/content/drive/My Drive/audio/avgListOut.npy")
popVar  = np.load("/content/drive/My Drive/audio/popVarListOut2.npy")
popAvg  = np.load("/content/drive/My Drive/audio/popAvgListOut2.npy")
print(popVar)
print(len(popVar))
print(progVar)
print(len(progVar))
f = plt.figure()
ax = f.add_subplot(111)
ax.scatter(np.array(progAvg), np.array(progVar), c='r',label="PROG")
ax.scatter(np.array(popAvg), np.array(popVar)+1e-8, c='b',label="NOT PROG")
ax.scatter(np.mean(np.array(progAvg)), np.mean(np.array(progVar)), c='r',marker='+', label="PROG_MEAN")
ax.scatter(np.mean(np.array(popAvg)), np.mean(np.array(popVar)), c='b',marker='+', label="NOT_PROG_MEAN")
plt.title("Variance of Tempo over 20s song segments")
plt.xlabel("Total AVG BPM")
plt.ylabel("BPM Variance")
plt.legend()
plt.show()'''
print(progSongs)
print(len(progSongs))
progAvg = np.load("/content/drive/My Drive/audio/progAvgListOut.npy")
progVar = np.load("/content/drive/My Drive/audio/progVarListOut.npy")
print(len(progAvg))
print(len(progVar))
log("PROG", progSongs, progAvg, progVar)


['01 - Autumn Lords.mp3', '01 - Birds of Fire.mp3', '01 - Luminol.mp3', '01 - My Spirit Will Go On.mp3', '01 - Song Within A Song.mp3', '01 - UFO.mp3', '01 Close to the Edge I. The Solid Time of Change II. Total Mass Retain III.mp3', '01 Hortz Fur Dehn Stekehn West.mp3', '01 Jordrok.mp3', '01 Roundabout.mp3', '01. Hazard Profile Part One.mp3', '01. Vert.mp3', '01_-_ace_of_wands_320_lame_cbr.mp3', '01Al Di Meola - Flight over Rio.mp3', '01-Hocus Pocus.mp3', '01-hymn_of_the_seventh_galaxy_320_lame_cbr.mp3', '01-Sospesi Nell_Incredibile.mp3', '01-The Bryden 2-Step (For Amphibians), Pt.1.mp3', '01-The runaway.mp3', '01-Zarathustra.mp3', '02 - Arbeit Macht Frei.mp3', '02 - Odyssey.mp3', '02 Can You Understand.mp3', '02 Master Of Puppets (LP Version).mp3', '02 Scorched Earth.mp3', '02 Willie The Pimp.mp3', '02. The Road Of Bones.mp3', '02_Dogs.mp3', '03 - Suenos De Maniqui.mp3', '03. Omen.mp3', '03_Anesthetize.mp3', '03-genesis-firth_of_fifth.mp3', '03Uriah Heep - July Morning.mp3', '04 - Th